In [13]:
from importlib import reload
import datasets
import torch

rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [14]:
import src.models.components.sign_language_net
reload(src.models.components.sign_language_net)
from src.models.components.sign_language_net import SignLanguageNet


model = SignLanguageNet()

src_tensor = torch.rand(1, 3, 224, 224)
out = model(src_tensor)

out.shape

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds